# 引言

在本次IR实验中，请运用所学的知识，为搜索引擎构建一个简单的索引和检索系统，包括：

1. [构建非压缩索引及检索实现](#构建非压缩索引及检索实现) 在stanford.edu网页语料库上构建未压缩索引，并实现布尔查询检索。
2. [构建压缩索引](#构建压缩索引) 使用可变长度编码在同一个语料库上构建压缩索引，并实现布尔查询。
3. [附加题](#其他压缩方法实现(选做))


## 实验要求及提交说明

1\. 本次实验将于学期结束前提交，具体时间将提前至少2周发布。

2\. 实验项目中包括：

**本实验文档PA1-skeleton.ipynb**

**实验环境安装说明README.md**

**测试语料库文件夹toy-data**

**对完整语料库的查询文件dev_queries**

**完整的语料库压缩文件**

**其他配置文件**

实验推荐安装Anaconda平台（python3版本），具体安装方法请参考网上教程。

2\. 实验需要提交内容包括：

**一份实验报告**（文件名：姓名_学号）

**通过dev_queries文件夹内的查询得到的查询结果文件，命名为dev_out，文件夹内命名规则与dev_queries保持一致**

**实验项目所有内容的压缩文件**（注意：不要任意修改文件名，便于助教进行测试）

其中，实验报告的内容需要包括以下内容：

(1) 需要补充的关键代码及说明；

(2) 所有存在输出结果的结果输出截图（包括测试部分）；

(3) （选做）附加部分的代码说明及分析；

(4) 实验总结。

3\. 请将以上所有提交内容打包到一个压缩文件中，命名为IREXP_姓名_学号，发送至邮箱：irdm_ustc@163.com 。

4\. 本实验参考Stanfod相关课程实验，故以下内容以英文为主。由于本学年第一次新增IR部分实验，如对实验内容有任何问题，欢迎与助教讨论或在QQ群内讨论，助教邮箱：gxq13@mail.ustc.edu.cn 。

## Setup

In [2]:
#Load the tee magic which saves a copy of the cell when executed
%reload_ext autograding_magics

The `submission` folder will contain all the files to be submitted.

In [3]:
import os
try: 
    os.mkdir('submission')
except FileExistsError:
    pass

In [4]:
%%tee submission/imports.py

# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

Overwriting submission/imports.py


# Corpus （语料库）

The corpus you will be working with for this assignment contains webpages from the stanford.edu domain. The data for this assignment is available as a .zip file at: http://web.stanford.edu/class/cs276/pa/pa1-data.zip. The following code puts the data folder under the current directory. The total size of the corpus is about 170MB.
（该文件已提前下载在实验文件中）

In [5]:
import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
# urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()

KeyboardInterrupt: 

The index will be stored in `output_dir` and `tmp` will contain some temporary files for toy indices.

In [6]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

There are 10 sub-directories (named 0-9) under the data directory.

In [7]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

Each file under the sub-directory is the content of an individual webpage. You should assume that the individual file names are unique within each sub-directory, but not necessarily the case across sub-directories (*i.e.,* the full path of the files are unique).

In [8]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

All the HTML info has been stripped off from those webpages, so they only contain space-delimited words. You should not re-tokenize the words. Each consecutive span of non-space characters consists of a word token in the corpus.

In [9]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

You will also find a small corpus included with the assignment in the folder `toy-data`. We will use this toy dataset to test our code before running on the complete dataset.

In [10]:
toy_dir = 'toy-data'

# 构建非压缩索引及检索实现

The first part of this assignment is to build an inverted index of this corpus, and implement Boolean conjunctive queries. **In particular, you need to implement the blocked sort-based indexing (BSBI) algorithm described in [Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf) of the textbook.** While we quote some fundamentals from the textbook, we strongly suggest that you read Section 4.2 in more detail to fully understand the BSBI algorithm.

实验的第一部分是建立这个语料库的倒排索引，并实现布尔查询（包括实现教材4.2节中描述的BSBI算法）。


> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

In this assignment, you will be implementing methods for large collections that require the use of secondary storage (*i.e.,* disk).

## IdMap

Again quoting from Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

Let us first build a helper class `IdMap`, which maps strings to numeric ids (and vice-versa). We will need this to bijectively map term to termID and doc to docID.

Fill in the functions `_get_str` and `_get_id` in the following code. The only interface to this class is provided by `__getitem__` which gets the correct mapping depending on the type of the key. (See [these docs](https://docs.python.org/3.7/reference/datamodel.html#special-method-names) if you are unfamiliar with special functions like `__getitem__` ). Further, to simplify code later on, we'll also add some logic to add a string if it doesn't already exist in the map.   

We will use a dictionary for efficient access from strings to numeric ids, and we will use a list for efficient access (and storage) from numeric ids to strings.

You might also find this [short tutorial](https://www.omkarpathak.in/2018/04/11/python-getitem-and-setitem/) useful to get started with special (a.k.a magic) methods


In [11]:
%%tee submission/id_map.py
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        if i > len(self.id_to_str) - 1:
            pass
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.id_to_str:
            self.id_to_str.append(s)
            self.str_to_id[s] = len(self.id_to_str) - 1
        return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

Overwriting submission/id_map.py


Make sure it passes the following simple test cases

In [12]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

Going forward we will expect you to write your own test cases to make sure parts of your program are working as expected.

## Encoding Postings List as bytearrays

In order to write and read lists of postings (docIDs) efficiently from the disk, we store them as bytearrays. We've provided an implementation of `UncompressedPostings` class which contains static encode and decode functions. In the next task you'll be required to implement compressed versions with the same inferface. 

For reference:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [13]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

To illustrate how it works, run the following cell

In [14]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## Inverted Index on Disk

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

In this section we provide a base class `InvertedIndex` which would be subsequently subclassed into `InvertedIndexWriter`, `InvertedIndexIterator` and `InvertedIndexMapper`. While typically `cPickle` is used for serialization in Python, it does not support partial reads and writes, which is why we are rolling out our own custom solution.

In [15]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

Since we are interacting with a file on disk, we provide `__enter__` and `__exit__` functions, which allow using the `with` statement just like you would with file IO in python (Refer to the [official documentation for context managers](https://docs.python.org/3/library/contextlib.html)).

Here is an example of how to use the `InvertedIndexWriter` context manager:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

If you are unfamiliar with context managers in python, we would recommend checking out these tutorials ([1](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/), [2](http://arnavk.com/posts/python-context-managers/)), although it isn't necessary to be able to understand all of it.

## Indexing

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

You should treat each sub-directory as a block, and only load one block in memory at a time when you build your index (note: you will be penalized if you construct the index by loading blocks of larger than one directory in memory at once). Note that we are abstracting away from the operating systems meaning of blocks. You can assume that each block is small enough to be stored in memory.

In this section, we introduce the class `BSBIIndex` which we will progressively build. The `index` function provides the skeleton for BSBI and your job is to implement `parse_block`, `invert_write` and `merge` functions in subsequent sections.

In [16]:

# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            print(block_dir_relative)
            td_pairs = self.parse_block(block_dir_relative)
            print(td_pairs)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### Parsing

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

You should treat each sub-directory as a block; `parse_block` expects the path to the sub-directory which is the block. You should assume that the individual file names are unique within each sub-directory, but not necessarily the case across sub- directories (i.e., the full path of the files are unique).

_NB - We are inheriting `BSBIIndex` from `BSBIIndex`, which is just a simple way of adding a new method to an existing class. Please do not be confused, it is simply used as a pedagogical tool to split a class definition within a jupyter notebook and there is no other magic happening here._

In [17]:
%%tee submission/parse_block.py
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        td_pairs = []
        block_to_parse = os.path.join(self.data_dir, block_dir_relative)
        docs = os.listdir(block_to_parse)
        parsed = 0
        for doc in docs:
            doc = os.path.join(block_to_parse, doc)
            parsed = parsed + 1
            if parsed % 1000 == 0:
                print(parsed)
            doc_id = self.doc_id_map[doc]
            td_pairs.extend(self._parse_file(doc, doc_id))
        return td_pairs
        ### End your code
        
    def _parse_file(self, doc, doc_id):
        td_pair = []
        with open(doc, 'r') as fp:
            content = fp.read()
        import re
        content = re.sub('[?!,.]', '', content)
        content = re.sub('\s+', ' ', content)
        content = content.strip().split(' ')
        for term in content:
            term_id = self.term_id_map[term]
            td_pair.append((term_id, doc_id))
        return td_pair

Overwriting submission/parse_block.py


See if the function works as expected on the toy data.

In [18]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [19]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 1), (4, 1), (5, 1), (6, 1), (3, 1)]

Does the `parse_block` method work as expected on a block of the toy data? Write some tests to make sure that is indeed the case (for example, you should make sure that a given word gets the same id each time it appears).

In [20]:
### Begin your code
t1 = BSBI_instance.parse_block('1')
t2 = BSBI_instance.parse_block('2')
print(t1, t2)
### End your code

[(7, 2), (8, 2), (9, 2), (3, 2), (10, 2), (9, 2), (3, 2), (11, 2), (10, 3), (10, 3), (10, 3)] [(0, 4), (1, 4), (2, 4), (3, 4), (4, 5), (4, 5), (5, 5), (6, 5), (3, 5)]


### Inversion

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

In this section we add the function `invert_write` which does exactly this. 

However, we first need to implement the class `InvertedIndexWriter`. This class provides an append function, just like a list, however the postings_list isn't stored in memory but is directly written to disk.

In [21]:
%%tee submission/inverted_index_writer.py
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        if self.terms == []:
            start_position_in_index_file = 0
        else:
            start_position_in_index_file = self.postings_dict[self.terms[-1]][0] + self.postings_dict[self.terms[-1]][2]
        number_of_postings_in_list = len(postings_list)
        postings_list = self.postings_encoding.encode(postings_list)
        length_in_bytes_of_postings_list = len(postings_list)
        self.postings_dict[term] = (start_position_in_index_file, number_of_postings_in_list, length_in_bytes_of_postings_list)
        self.terms.append(term)
        # content = self.index_file.read() + postings_list
        # print(self.index_file.read(), self.postings_encoding.decode(content))
        self.index_file.write(postings_list)
        ### End your code

Overwriting submission/inverted_index_writer.py


Even though we haven't written the classes which read the index, we can still test the implementation as follows. Make sure you pass the asserts before moving forward

In [22]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

Now we implement `invert_write`, which takes in the td_pairs created with parse_block. We use `InvertedIndexWriter` to write them to disk.

In [23]:
%%tee submission/invert_write.py
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        postings_lists = {}
        for td_pair in td_pairs:
            if td_pair[0] not in postings_lists:
                postings_lists[td_pair[0]] = [td_pair[1]]
            else:
                if td_pair[1] not in postings_lists[td_pair[0]]:
                    postings_lists[td_pair[0]].append(td_pair[1])
        keys = list(postings_lists.keys())
        keys.sort()
        for term in keys:
            index.append(term, postings_lists[term])
        ### End your code

Overwriting submission/invert_write.py


To test this on the toy data, we can read a block and see what the inverted index contains. Write some tests like you saw for the `InvertedIndexWriter`.

In [24]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')
with InvertedIndexWriter('test2', directory='tmp/') as index:
    td_pairs = BSBI_instance.parse_block('0')
    BSBI_instance.invert_write(td_pairs, index)
    index.index_file.seek(0)
    assert index.terms == [0,1,2,3,4,5,6], "terms sequence incorrect"
    assert index.postings_dict[0] == (0, 1, len(UncompressedPostings.encode([0]))), "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [0,0,0,0,1,1,1,1], "postings on disk incorrect"
### End your code

### Merging

> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

The iterator paradigm in Python lends itself naturally to our need of maintaining a small read buffer. We can iterate through the file while reading just one postings list at a time from the disk. We subclass `InvertedIndex` into `InvertedIndexIterator` which does the iteration bit. 

In [25]:
%%tee submission/inverted_index_iterator.py
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.foot = 0
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        if self.foot >= len(self.terms):
            return 0
        term = self.terms[self.foot]
        self.foot = self.foot + 1
        start = self.postings_dict[term][0]
        nums = self.postings_dict[term][1]
        length = self.postings_dict[term][2]
        self.index_file.seek(start, 0)
        posting_list = self.index_file.read(length)
        return self.postings_encoding.decode(posting_list)
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

Overwriting submission/inverted_index_iterator.py


Let's test this by first creating an index with the `InvertedIndexWriter` and then iterating through it. Write a few small test cases to see if it works. At the very least, you should write a test which manually constructs a small inverted index, writes them to disk with an `InvertedIndexWriter`, then uses an `InvertedIndexIterator` to iterate over the inverted index.

In [26]:
### Begin your code
with InvertedIndexWriter('testIterator', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.append(5, [3, 5, 1, 2])
    index.index_file.seek(0)
    
with InvertedIndexIterator('testIterator', directory='tmp/') as iters:
    print(iters.__next__())
    print(iters.__next__())
    print(iters.__next__())
    print(iters.__next__())
### End your code

[2, 3, 4]
[3, 4, 5]
[3, 5, 1, 2]
0


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

We'll use the `InvertedIndexIterator` to do the reading part and `InvertedIndexWriter` to write the merged postings list. 

Note that we've been opening each inverted index using `with` statement, but now we need to do that programmatically. You can see that we've used `contextlib` ([documentation](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack)) for that in the provided `index` function of `BSBIIndex` base class. Your task is to **write the logic** for merging *opened* `InvertedIndexIterator` objects and writing one postings list at a time into into a single `InvertedIndexWriter` object. 

Since we know that the postings are sorted, we can appropriately merge them in sorted order in linear time. In fact `heapq` ([documentation](https://docs.python.org/3.0/library/heapq.html)) is a standard python module that provides an implementation of the heap queue algorithm. In particular it contains a `heapq.merge` utility function which merges multiple sorted inputs into a single sorted output and returns an iterator over the sorted values. Not only can this be handy with merging postings lists, but also with merging the inverted indices. 

To get you started on using the `heapq.merge` function, we've provided a sample usage of the function. The two lists contain animals/birds sorted by their average life span. We want to merge the two lists.

In [27]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


Notice the use of `*` to unpack `lifespan_lists` as arguments and `lambda` function to reprsent the key to perform the soring with. If you are unfamiliar you can check documentation ([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions)) or tutorials ([unpacking lists](https://www.geeksforgeeks.org/packing-and-unpacking-arguments-in-python/), [lambda_expressions](https://www.afternerd.com/blog/python-lambdas/))

In [28]:
%%tee submission/merge.py

import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        feet = [0] * len(indices)
        parsed = 0
        while True:
            ret = []
            term, selected, feet, parsed = self._take_smallest(indices, feet)
            if parsed >= len(indices):
                break
            postings_lists = self._get_postings_lists(indices, selected)
            for merged_item in heapq.merge(*postings_lists):
                ret.append(merged_item)
            # print(term, ret)
            merged_index.append(term, ret)
        ### End your code
        
    def _take_smallest(self, indices, feet):
        smallest = None
        selected = []
        parsed = 0
        for i in range(len(feet)):
            if feet[i] == -1:
                parsed = parsed + 1
                continue
            if len(indices[i].terms) <= feet[i]:
                parsed = parsed + 1
                feet[i] = -1
            else:
                if smallest is None:
                    smallest = indices[i].terms[feet[i]]
                    selected = [i]
                else:
                    if smallest > indices[i].terms[feet[i]]:
                        smallest = indices[i].terms[feet[i]]
                        selected = [i]
                    elif smallest == indices[i].terms[feet[i]]:
                        selected.append(i)
        new_feet = feet
        for sel in selected:
            new_feet[sel] = new_feet[sel] + 1
        return smallest, selected, new_feet, parsed
    
    def _get_postings_lists(self, indices, selected):
        postings_lists = []
        for sel in selected:
            postings_lists.append(indices[sel].__next__())
        return postings_lists

Overwriting submission/merge.py


First make sure it works without errors on toy data

In [29]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

0
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 1), (4, 1), (5, 1), (6, 1), (3, 1)]
1
[(7, 2), (8, 2), (9, 2), (3, 2), (10, 2), (9, 2), (3, 2), (11, 2), (10, 3), (10, 3), (10, 3)]
2
[(0, 4), (1, 4), (2, 4), (3, 4), (4, 5), (4, 5), (5, 5), (6, 5), (3, 5)]


Now lets index the full corpus

In [149]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



9
1000
2000
3000
4000
5000
6000
7000
8000
9000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



If you're having errors with just the merging part of it, you can use the following code to debug it. 

In [338]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## Boolean conjunctive retrieval

We'll be implementing a function `retrieve` to BSBIIndex, which given a query string consisting of space separated tokens, returns a list of documents that contain each of the tokens in the query. However, we do not want to be iterating through the index nor loading the entire index to find the relevant terms. 

First we'll implement `InvertedIndexMapper` which subclasses `InvertedIndex` to add functionality for retrieving postings corresponding to a particular term by seeking to that location in the file.

In [30]:
%%tee submission/inverted_index_mapper.py
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        if term not in self.postings_dict:
            return []
        start_position = self.postings_dict[term][0]
        byte_length = self.postings_dict[term][2]
        self.index_file.seek(start_position)
        posting_list_byte = self.index_file.read(byte_length)
        posting_list = self.postings_encoding.decode(posting_list_byte)
        return posting_list
        ### End your code

Overwriting submission/inverted_index_mapper.py


Write a few tests to check your `_get_postings_list` implementation.

In [31]:
### Begin your code
with InvertedIndexMapper('BSBI', directory='output_dir/') as index:
    print(index[140])
### End your code

[0, 2, 3, 181, 252, 253, 261, 262, 267, 270, 271, 272, 273, 276, 281, 282, 290, 291, 463, 464, 477, 479, 481, 482, 484, 488, 489, 491, 614, 620, 621, 652, 765, 957, 1056, 1058, 1075, 1117, 1137, 1250, 1420, 1455, 1563, 1635, 1644, 1748, 1751, 1838, 1847, 1890, 1891, 1895, 1897, 1915, 1931, 1932, 1934, 1942, 2201, 2592, 2594, 2596, 2597, 2598, 2599, 2600, 2601, 2680, 3311, 3329, 3374, 3417, 3430, 3648, 3657, 3660, 3878, 3974, 4000, 4001, 4002, 4003, 4004, 4005, 4006, 4007, 4008, 4009, 4010, 4011, 4012, 4013, 4014, 4015, 4016, 4017, 4018, 4019, 4020, 4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031, 4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040, 4058, 4059, 4066, 4067, 4071, 4075, 4076, 4077, 4078, 4079, 4080, 4122, 4125, 4140, 4142, 4143, 4211, 4217, 4291, 4296, 4310, 4317, 4351, 4366, 4368, 4369, 4370, 4371, 4372, 4373, 4374, 4375, 4376, 4377, 4378, 4379, 4380, 4381, 4382, 4384, 4386, 4388, 4389, 4390, 4391, 4394, 4395, 4396, 4397, 4398, 4399, 4400, 4401, 4402, 4

Now that we can obtain postings lists corresponding to the query terms, we want to intersect them. However, à la merging, we can use the sorted-ness of these lists to intersect them very efficiently. Here we'll implement `sorted_intersect` which takes two sorted lists and return a sorted intersection of the elements.  

In [32]:
%%tee submission/sorted_intersect.py
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    result = []
    for elem in list1:
        if elem in list2:
            result.append(elem)
    return result
    ### End your code

Overwriting submission/sorted_intersect.py


Write a few simple test cases to make sure it works correctly

In [34]:
### Begin your code
list1 = [1,3,5,8,10]
list2 = [2,4,8,10,15]
print(sorted_intersect(list1, list2))
### End your code

[8, 10]


Now we're ready to write the `retrieve` function using `InvertedIndexMapper` and `sorted_intersect`

In [35]:
%%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        import re
        with InvertedIndexMapper('BSBI', directory='output_dir/') as mapper:
            query = re.sub('[,.!?]', '', query)
            query = re.sub('\s+', ' ', query)
            tokens = query.strip().split(' ')
            result = None
            for token in tokens:
                token = self.term_id_map[token]
                if result is None:
                    result = mapper[token]
                else:
                    result = sorted_intersect(result, mapper[token])
            for i in range(len(result)):
                result[i] = self.doc_id_map[result[i]]
            return result
        ### End your code

Overwriting submission/retrieve.py


Let us test if our index works on the real corpus with a simple query

In [36]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

['pa1-data\\1\\cs276.stanford.edu_',
 'pa1-data\\1\\cs276a.stanford.edu_',
 'pa1-data\\3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford

You can also test if one of these pages truly contains the query terms by reading a file as follows

In [37]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

Let's test on the dev queries to see if it works alright. 

In [39]:
try:
    os.mkdir('dev_output')
except:
    pass
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out', 'wb') as o:
            # reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            # assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
            pkl.dump(my_results, o)
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


If an assert is failing you can compare the difference in outputs as follows

In [ ]:
set(my_results) - set(reference_results)

In [ ]:
set(reference_results) - set(my_results)

Make sure you write your own queries test for all kinds of corner cases like terms which have never occurred in the corpus, or terms that are very frequent and might slow down the merge. 

# 构建压缩索引

In this section, you should build a compressed index using gap encoding with variable byte encoding for each gap.

Here are some things about python you'll find useful. 

In [40]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


Fill in the following `CompressedPostings` class which we shall use as a drop-in replacement for `UncompressedPostings`. You should refer to [Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) of the textbook for understanding how gap encoding with variable byte encoding for each gap is done. 

In [41]:
%%tee submission/compressed_postings.py
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def prepend(bts, number):
        number = number % 128
        number = number.to_bytes(length=1, byteorder='little')
        if bts is None:
            bts = number
        else:
            bts = number + bts
        return bts
    
    @staticmethod
    def encode_number(number):
        bts = None
        while True:
            bts = CompressedPostings.prepend(bts, number)
            if number < 128:
                break
            number = number // 128
        bt_modified = (bts[len(bts)-1] + 128).to_bytes(length=1, byteorder='little')
        bts = bts[0:len(bts)-1] + bt_modified
        return bts
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        bytestream = None
        for number in postings_list:
            if bytestream is None:
                bytestream = CompressedPostings.encode_number(number)
            else:
                bytestream += CompressedPostings.encode_number(number)
        return bytestream
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        numbers = []
        n = 0
        for i in range(len(encoded_postings_list)):
            temp = encoded_postings_list[i]
            if temp < 128:
                n = 128 * n + temp
            else:
                n = 128 * n + (temp - 128)
                numbers.append(n)
                n = 0
        return numbers
        ### End your code


Overwriting submission/compressed_postings.py


First, write some test cases for any helper methods that you might have implemented

In [42]:
### Begin your code
postings_list = [1,3,6,8,10,6,2,78,45,26,12200]
a = CompressedPostings.encode(postings_list)
print(a)
b = CompressedPostings.decode(a)
print(b)
### End your code

b'\x81\x83\x86\x88\x8a\x86\x82\xce\xad\x9a_\xa8'
[1, 3, 6, 8, 10, 6, 2, 78, 45, 26, 12200]


We've provided a simple helper function for you to test whether an encoded postings list is decode correctly.

In [43]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

Write a few test cases to make sure the postings list compression and decompression is being done correctly

In [44]:
### Begin your code
postings_list = [1,3,6,8,10,6,2,78,45,26,122]
a = CompressedPostings.encode(postings_list)
print(a)
b = CompressedPostings.decode(a)
print(b)
### End your code

b'\x81\x83\x86\x88\x8a\x86\x82\xce\xad\x9a\xfa'
[1, 3, 6, 8, 10, 6, 2, 78, 45, 26, 122]


Now let us create a new output directory and use the `CompressedPostings` to create a compressed index

In [45]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [370]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



9
1000
2000
3000
4000
5000
6000
7000
8000
9000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['pa1-data\\1\\cs276.stanford.edu_',
 'pa1-data\\1\\cs276a.stanford.edu_',
 'pa1-data\\3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 'pa1-data\\4\\nlp.stanford

As before, let us now test on the dev queries to see if it works alright

In [47]:
try:
    os.mkdir('dev_output_compressed')
except:
    pass
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output_compressed/' + str(i) + '.out', 'wb') as o:
            # reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            # assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
            pkl.dump(my_results, o)
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


## Code submission

Now you are ready to submit the code part of your assignment. Refer to [submission instructions section](#Submission-instructions). 

# 其他压缩方法实现(选做)

Implement one more index compression method by filling in the `encode` and `decode` methods in `ECCompressedPostings`. 

Here are a few directions  
1. [Section 5.4](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) in the textbook points to a few techniques (e.g., **gamma-encoding**) and related publications. 
2. You can also look into **Delta Encoding**, a compression technique very similar to Gamma Encoding, which can achieve an even better compression rate. 
3. Also, [ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) is an interesting encoding technique that you could also consider implementing.

You should store each postings list is stored in a multiple of bytes (rather than bits). The positions and lengths in index file are in multiples of bytes. 

In [76]:
%%tee submission/ec_compressed_postings.py
class ECCompressedPostings:
    ### gamma-encoding
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def getLength(number):
        length = 0
        while number > (2 ** length):
            length += 1
        if number == 2 ** length:
            return length
        return length - 1
    
    @staticmethod
    def getOffset(number, length):
        return number - 2 ** length
    
    @staticmethod
    def convertDecimal(length, offset):
        coefficient = 2 ** (length + 1)
        lengthDecimal = 2 ** length - 1
        return coefficient * lengthDecimal + offset
        
    @staticmethod    
    def prepand(bts, decimal):
        bt = decimal.to_bytes(length=1, byteorder='little')
        if bts is None:
            return bt
        return bt + bts
    
    @staticmethod
    def encodeDecimal(decimal):
        bts = None
        while decimal >= 256:
            bts = ECCompressedPostings.prepand(bts, decimal % 256)
            decimal = decimal // 256
        return ECCompressedPostings.prepand(bts, decimal)
        
    @staticmethod
    def encode_number(number):
        length = ECCompressedPostings.getLength(number)
        offset = ECCompressedPostings.getOffset(number, length)
        decimal = ECCompressedPostings.convertDecimal(length, offset)
        bts = ECCompressedPostings.encodeDecimal(decimal)
        return bts
    
    @staticmethod
    def decodeDelimiter(number, status):
        if status == 1:
            i = 7
            ret, flag = 0, 0
            while i >= 0:
                if flag == 0 and number & (2 ** i) != 0:
                    ret += 1
                    flag = 1
                    start_point = i
                elif flag == 1 and number & (2 ** i) != 0:
                    ret += 1
                elif flag == 1 and number & (2 ** i) == 0:
                    return start_point, ret
                i -= 1
            else:
                raise ValueError('number is \x00, should not occur')
        for i in range(7, -1, -1):
            if number & (2 ** i) == 0:
                return 7, 7 - i
        else:
            raise ValueError('number is \xff, no delimiter found')
            
    @staticmethod
    def decodeOffset(encoded_postings_list, start_point, resLen, length, currentIdx):
        resOffset = encoded_postings_list[currentIdx] & (2 ** (start_point - resLen) - 1)
        offsetLen = length + resLen - 7
        if start_point != 7:
            offsetLen = resLen
        # assert offsetLen % 8 == 0, "offsetLen is {}".format(offsetLen)
        exceedIdx = offsetLen // 8
        offset = resOffset
        for i in range(1, exceedIdx+1):
            offset = offset * 256 + encoded_postings_list[currentIdx + i]
        return offset, currentIdx + exceedIdx + 1
    
    @staticmethod
    def decodeDecimal(length, offset):
        return 2 ** length + offset
    
    @staticmethod
    def fillGap(numbers):
        newNumbers = []
        base = 0
        for gap in numbers:
            base = base + gap
            newNumbers.append(base)
        return newNumbers
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        beforehand = 0
        bytestream = None
        for number in postings_list:
            gap = number - beforehand
            encodedBytes = ECCompressedPostings.encode_number(gap)
            if bytestream is None:
                bytestream = encodedBytes
            else:
                bytestream += encodedBytes
            beforehand = number
        return bytestream
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        from math import log
        numbers = []
        length = 0
        i = 0
        status = 1
        while i < len(encoded_postings_list):
            if encoded_postings_list[i] == 0:
                numbers.append(1)
                i += 1
                continue
            temp = log((encoded_postings_list[i] + 1), 2)
            if int(temp) == temp:
                length += ECCompressedPostings.getLength(encoded_postings_list[i] + 1)
                i += 1
                status = 2
            else:
                start_point, resLen = ECCompressedPostings.decodeDelimiter(encoded_postings_list[i], status)
                length += resLen
                offset, i = ECCompressedPostings.decodeOffset(encoded_postings_list, start_point, resLen, length, i)
                number = ECCompressedPostings.decodeDecimal(length, offset)
                numbers.append(number)
                length = 0
                status = 1
        numbers = ECCompressedPostings.fillGap(numbers)
        return numbers
        ### End your code

Overwriting submission/ec_compressed_postings.py


In [97]:
### Begin your code
postings_list = [1, 2, 8798696, 1112899000]
a = ECCompressedPostings.encode(postings_list)
print(a)
b = ECCompressedPostings.decode(a)
print(b)
### End your code

b'\x00\x00\x7f\xff\xff\x06A\xe6\x1f\xff\xff\xff\x81\xcf;\xd0'
[1, 2, 8798696, 1112899000]


**Write a description of your index compression method as well as a discussion of the space/speed trade-off of this additional compression method, in comparison to what we implemented in previous tasks.**

> Gamma-encoding

In [94]:
import time
postings_list = []
number = 0
for j in range(1, 200000):
    if j > 100:
        j = 1
    number += j
    postings_list.append(number)

# uncompressed
start = time.time()
unpressed = UncompressedPostings.encode(postings_list)
length_unpressed = len(unpressed)
assert postings_list == UncompressedPostings.decode(unpressed)
end = time.time()
print(length_unpressed)
print('length:{}, {}s for unpressed coding'.format(length_unpressed, end-start))

# variable coding
start = time.time()
unpressed = CompressedPostings.encode(postings_list)
length_unpressed = len(unpressed)
assert postings_list == CompressedPostings.decode(unpressed)
end = time.time()
print(length_unpressed)
print('length:{}, {}s for variable coding'.format(length_unpressed, end-start))

# gamma coding
start = time.time()
unpressed = ECCompressedPostings.encode(postings_list)
length_unpressed = len(unpressed)
assert postings_list == ECCompressedPostings.decode(unpressed)
end = time.time()
print(length_unpressed)
print('length:{}, {}s for gamma coding'.format(length_unpressed, end-start))

799996
length:799996, 0.013950347900390625s for unpressed coding
588549
length:588549, 2.571326494216919s for variable coding
200084
length:200084, 1.0661489963531494s for gamma coding
